In [1]:
! wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

--2024-06-20 19:11:13--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-20 19:11:13 (9.18 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [4]:
! wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-06-20 19:15:51--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.04s   

2024-06-20 19:15:51 (16.2 MB/s) - ‘documents.json’ saved [658332/658332]



In [1]:
import minsearch
import json
from openai import OpenAI

In [ ]:
client = OpenAI()

In [2]:
with open('documents.json', 'r') as f:
    doc_raws = json.load(f)

In [3]:
documents = []
for course_dict in doc_raws:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
index = minsearch.Index(text_fields = ['question', 'text', 'section'],
                         keyword_fields = ['course']
                        )
index.fit(documents)

In [87]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}
    result  = index.search(query = query,
                 boost_dict = boost,
                 num_results = 5
                )
    return result
    
def build_prompt(query, search_result):

    context = ""
    for doc in search_result:
        context =context + f"section :{doc['section']} \nquestion:{doc['question']}\nanswer:{doc['text']}\n\n"
    
    prompt_template = """
    You are a course teaching assistant. Answer the given QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}\n
    CONTEXT: {context}
    """.strip()
    prompt = prompt_template.format(question = query, 
                                    context = context).strip()
    return prompt

def llm (prompt):
    response = client.chat.completions.create(
    model = 'gpt-4o',
    messages =[{'role':'user', 'content': prompt}]
    )
    return response.choices[0].message.content

In [92]:
def rag(query):
    results = search(query)
    prompt = build_prompt(query, results)
    answer = llm(prompt)
    return answer

In [93]:
query = "Why am I not getting interviews?"
rag(query)

'Based on the provided CONTEXT, there is no specific answer to your question about not getting interviews. The CONTEXT mainly discusses technical course-related questions and issues related to assignments and project configurations.\n\nHowever, general advice can be given:\n\n1. **Review and Improve Your Resume/CV**: Typos, lack of relevant experiences, or a poor format can be reasons for not getting callbacks.\n2. **Tailor Applications**: Ensure that each application is tailored to the specific job and company.\n3. **Networking**: Sometimes personal connections or referrals can help bypass some of the initial screening processes.\n4. **Skills and Experience Alignment**: Ensure your skills and experiences align closely with the job description.\n\nIf you need further assistance, it might be helpful to seek advice from a career coach or mentor who can provide more personalized guidance.'